In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, roc_curve

from sklearn.svm import SVC
import seaborn as sns




In [ ]:
data = pd.read_csv('/content/tabular-actgan-65415ed27fe248f10ea57922-data.csv')

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
print(data.columns)

In [ ]:
X = data.drop('depression_diagnosis', axis=1)
y = data['depression_diagnosis']


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
cat_col = [ 'gender', 'ethnicity', 'marital_status', 'education_level',
       'employment_status', 'smoking_status']

In [ ]:
num_col = ['age', 'income', 'social_support',
       'stress_level', 'sleep_quality', 'exercise_frequency',
       'alcohol_consumption', 'self_esteem_level',
       'loneliness_level', 'life_satisfaction', 'positive_emotions',
       'negative_emotions']

In [ ]:
bol_col = ['family_history' ,'drug_use',
       'depression_diagnosis', 'anxiety_diagnosis', 'suicidal_thoughts',
       'therapy_attendance', 'medication_usage']

In [ ]:
# Apply Label Encoding to categorical columns
label_encoder_dict = {}
for col in cat_col:
    label_encoder = LabelEncoder()
    X_train[col] = label_encoder.fit_transform(X_train[col])
    X_test[col] = label_encoder.transform(X_test[col])
    label_encoder_dict[col] = label_encoder

In [ ]:
# Standardize numerical features
scaler = StandardScaler()
X_train[num_col] = scaler.fit_transform(X_train[num_col])
X_test[num_col] = scaler.transform(X_test[num_col])

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif

# Assume X_train is your feature matrix, and y_train is the target variable
selector = SelectKBest(score_func=f_classif, k=10)
X_new = selector.fit_transform(X_train, y_train)

# Get the names of the top features
top_feature_indices = selector.get_support(indices=True)
top_feature_names = X_train.columns[top_feature_indices]

print("Top Feature Names:", top_feature_names)

In [ ]:
# Extract the top features from X_train
X_top_features = X_train[top_feature_names]

# Concatenate the top features with the target variable (y_train)
data_for_corr = pd.concat([X_top_features, y_train], axis=1)

# Calculate the correlation matrix
corr_matrix = data_for_corr.corr()

# Set up the matplotlib figure
plt.figure(figsize=(12, 10))

# Draw the heatmap using seaborn
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)

# Show the plot
plt.title("Correlation Matrix for Top Selected Features")
plt.show()

In [ ]:
top_feature_names = ['age', 'social_support', 'sleep_quality', 'anxiety_diagnosis',
       'suicidal_thoughts', 'therapy_attendance', 'medication_usage',
       'self_esteem_level', 'loneliness_level', 'positive_emotions']

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
from keras.layers import BatchNormalization

model = Sequential()
model.add(Dense(64, input_dim=X_train[top_feature_names].shape[1], activation='relu'))
model.add(BatchNormalization())
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))
from keras.layers import Dropout


# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train[top_feature_names], y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test[top_feature_names], y_test)
print(f'Neural Network Accuracy: {accuracy:.4f}')

# Save the model
model.save("depression_diagnosis.h5")


In [ ]:
# Save the model
model.save("suicidal_thoughts_model.h5")

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

# Assuming model is your trained neural network model
y_pred = model.predict(X_test[top_feature_names])
y_pred_binary = (y_pred > 0.5).astype(int)  # Assuming it's a binary classification task

# Print classification report
print("Neural Network Classification Report:")
print(classification_report(y_test, y_pred_binary))

# Print confusion matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_binary))